In [1]:
import csv 
import numpy as np
from numpy.linalg import inv
import random
import math
import sys
import pandas as pd

In [3]:
data = []
# 每一個維度儲存一種污染物的資訊
for i in range(18):
	data.append([])

n_row = 0
text = open('../data/train.csv', 'r', encoding='big5') 
row = csv.reader(text , delimiter=",")
for r in row:
    # 第0列沒有資訊
    if n_row != 0:
        # 每一列只有第3-27格有值(1天內24小時的數值)
        for i in range(3,27):
            if r[i] != "NR":
                data[(n_row-1)%18].append(float(r[i]))
            else:
                data[(n_row-1)%18].append(float(0))	
    n_row = n_row+1
text.close()

In [4]:
da = np.array(data)

In [6]:
train_data = pd.read_csv('../data/train.csv',encoding = 'big5')
train_data = train_data.values
train_data.shape

(4320, 27)

In [10]:
x = []
y = []
# 每 12 個月
for i in range(12):
    # 一個月取連續10小時的data可以有471筆
    for j in range(471):
        x.append([])
        # 18種污染物
        for t in range(18):
            # 連續9小時
            for s in range(9):
                x[471*i+j].append(data[t][480*i+j+s] )
        y.append(data[9][480*i+j+9])
x = np.array(x)
y = np.array(y)
print(x.shape)
print(y.shape)
# add square term
# x = np.concatenate((x,x**2), axis=1)

# add bias
x = np.concatenate((np.ones((x.shape[0],1)),x), axis=1)

(5652, 162)
(5652,)


In [14]:
x

array([[ 1. , 14. , 14. , ...,  2. ,  2. ,  0.5],
       [ 1. , 14. , 14. , ...,  2. ,  0.5,  0.3],
       [ 1. , 14. , 13. , ...,  0.5,  0.3,  0.8],
       ...,
       [ 1. , 17. , 18. , ...,  1.1,  1.4,  1.3],
       [ 1. , 18. , 19. , ...,  1.4,  1.3,  1.6],
       [ 1. , 19. , 18. , ...,  1.3,  1.6,  1.8]])

In [8]:
x.shape

(5652, 163)

In [9]:
w = np.zeros(len(x[0]))
l_rate = 10
repeat = 100000

In [ ]:
x_t = x.transpose()
s_gra = np.zeros(len(x[0]))

for i in range(repeat):
    hypo = np.dot(x,w)
    loss = hypo - y
    cost = np.sum(loss**2) / len(x)
    cost_a  = math.sqrt(cost)
    gra = np.dot(x_t,loss)
    s_gra += gra**2
    ada = np.sqrt(s_gra)
    w = w - l_rate * gra/ada
    print ('iteration: %d | Cost: %f  ' % ( i,cost_a))

In [32]:
# save model
np.save('model.npy',w)
# read model
w = np.load('model.npy')

In [33]:
test_x = []
n_row = 0
text = open('data/test.csv' ,"r")
row = csv.reader(text , delimiter= ",")

In [34]:
for r in row:
    if n_row %18 == 0:
        test_x.append([])
        for i in range(2,11):
            test_x[n_row//18].append(float(r[i]) )
    else :
        for i in range(2,11):
            if r[i] !="NR":
                test_x[n_row//18].append(float(r[i]))
            else:
                test_x[n_row//18].append(0)
    n_row = n_row+1
text.close()
test_x = np.array(test_x)

# add square term
# test_x = np.concatenate((test_x,test_x**2), axis=1)

# add bias
test_x = np.concatenate((np.ones((test_x.shape[0],1)),test_x), axis=1)

In [37]:
ans = []
for i in range(len(test_x)):
    ans.append(["id_"+str(i)])
    a = np.dot(w,test_x[i])
    ans[i].append(a)

filename = "predict.csv"
text = open(filename, "w+")
s = csv.writer(text,delimiter=',',lineterminator='\n')
s.writerow(["id","value"])
for i in range(len(ans)):
    s.writerow(ans[i]) 
text.close()

In [53]:
def get_err(file1, file2):
    y1 = pd.read_csv(file1).value.values
    y2 = pd.read_csv(file2).value.values
    y1,y2 = np.array(y1), np.array(y2)
    errors = np.sum(((y1 - y2)**2)/len(y1))**0.5  
    print(errors)

In [54]:
get_err('predict.csv','ans.csv')

6.769924445192698
